In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("CSVImport").getOrCreate()

25/06/15 16:14:50 WARN Utils: Your hostname, Vijays-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.71 instead (on interface en0)
25/06/15 16:14:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/15 16:14:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("/Users/vijayramaraju/Developer/Data Engineering/movieassist/data/raw/TMDB_movie_dataset_v11.csv", header=True, inferSchema=True)

In [4]:
df.show(10)

+------+--------------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|    id|               title|vote_average|vote_count|  status|release_date|   revenue|runtime|adult|       backdrop_path|   budget|            homepage|  imdb_id|original_language|      original_title|            overview|          popularity|         poster_path|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+------+--------------------+------------+----------+--------+------------+----------+-------+-----+--------------------+---------+--------------------+---------+-----------------+--------------------+-----------------

In [5]:
df = df.drop( "id", "revenue", "backdrop_path" , "budget" , "homepage" , "imdb_id" , "poster_path" )


In [6]:
df.dtypes

[('title', 'string'),
 ('vote_average', 'string'),
 ('vote_count', 'string'),
 ('status', 'string'),
 ('release_date', 'string'),
 ('runtime', 'string'),
 ('adult', 'string'),
 ('original_language', 'string'),
 ('original_title', 'string'),
 ('overview', 'string'),
 ('popularity', 'string'),
 ('tagline', 'string'),
 ('genres', 'string'),
 ('production_companies', 'string'),
 ('production_countries', 'string'),
 ('spoken_languages', 'string'),
 ('keywords', 'string')]

In [7]:
df.show(12)

+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|vote_average|vote_count|  status|release_date|runtime|adult|original_language|      original_title|            overview|          popularity|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           Inception|       8.364|     34495|Released|  2010-07-15|    148|False|               en|           Inception|"Cobb, a 

In [8]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, FloatType, StringType

df = (df
    .withColumn("vote_average", col("vote_average").cast(FloatType()) )
    .withColumn("vote_count", col("vote_count").cast(IntegerType()))
    .withColumn("release_date", col("release_date").cast(StringType()))
    .withColumn("runtime", col("runtime").cast(FloatType()))
)


In [9]:
df.show(10)

+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|vote_average|vote_count|  status|release_date|runtime|adult|original_language|      original_title|            overview|          popularity|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|           Inception|       8.364|     34495|Released|  2010-07-15|  148.0|False|               en|           Inception|"Cobb, a 

In [10]:
df = df.filter(col("popularity").cast(FloatType()).isNotNull())

In [11]:
df.show(10)

+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|vote_average|vote_count|  status|release_date|runtime|adult|original_language|      original_title|            overview|popularity|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        Interstellar|       8.417|     32571|Released|  2014-11-05|  169.0|False|               en|        Interstellar|The adventures of...|   140.241|Mankind

In [15]:
df.select("status").distinct().show(19)


+--------------------+
|              status|
+--------------------+
|            Released|
|             Rumored|
|       In Production|
|            Canceled|
|             Planned|
|     Post Production|
| 11 Loads) Nipple...|
+--------------------+



In [12]:
df.write \
    .mode("overwrite") \
    .option("header", True) \
    .csv("/Users/vijayramaraju/Developer/Data Engineering/movieassist/data/processed")

In [ ]:
# embeddings making

In [ ]:
# [title] ([release_date_year]) is a [genres] film rated [vote_average]/10 by [vote_count] viewers. 
# This [runtime]-minute [original_language] movie [status_description].

# Story: [overview] [tagline]

# Movie Profile: This [genres] film features [keywords] themes. The movie was produced by 
# [production_companies] in [production_countries] with dialogue in [spoken_languages]. 
# Popularity score: [popularity]. [adult_content_note]

In [16]:
from pyspark.sql.functions import col, year, when, format_string, to_date, lit

processed_df = spark.read.csv(
    "/Users/vijayramaraju/Developer/Data Engineering/movieassist/data/processed",
    header=True,
    inferSchema=True
)



In [17]:
processed_df.show(10)

+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|               title|vote_average|vote_count|  status|release_date|runtime|adult|original_language|      original_title|            overview|popularity|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|
+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|        Interstellar|       8.417|     32571|Released|  2014-11-05|  169.0|False|               en|        Interstellar|The adventures of...|   140.241|Mankind

In [18]:
string_cols = [c for c, t in processed_df.dtypes if t == 'string']
numeric_cols = [c for c, t in processed_df.dtypes if t in ['int', 'bigint', 'float', 'double']]
bool_cols = [c for c, t in processed_df.dtypes if t == 'boolean']

processed_df = processed_df.fillna('N/A', subset=string_cols)
processed_df = processed_df.fillna(0, subset=numeric_cols)
processed_df = processed_df.fillna(False, subset=bool_cols)

# Create derived columns for the document
# Extract year from release_date
processed_df = processed_df.withColumn("release_date_casted", to_date(col("release_date"), "yyyy-MM-dd"))
processed_df = processed_df.withColumn("release_date_year",
    when(col("release_date_casted").isNotNull(), year(col("release_date_casted"))).otherwise(1900)
)

# Create a description from the 'status' column
processed_df = processed_df.withColumn("status_description",
    when(col("status") == "Released", "has been released")
    .when(col("status") == "In Production", "is currently in production")
    .when(col("status") == "Post Production", "is in post-production")
    .otherwise("has an unknown status")
)

# Create a note about adult content
processed_df = processed_df.withColumn("adult_content_note",
    when(col("adult") == True, "Intended for adult audiences.")
    .otherwise("Suitable for all audiences.")
)


In [19]:
processed_df.show(14)

+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-----------------+------------------+--------------------+
|               title|vote_average|vote_count|  status|release_date|runtime|adult|original_language|      original_title|            overview|popularity|             tagline|              genres|production_companies|production_countries|    spoken_languages|            keywords|release_date_casted|release_date_year|status_description|  adult_content_note|
+--------------------+------------+----------+--------+------------+-------+-----+-----------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----

In [20]:

# Define the document template
document_template = """%s (%s) is a %s film rated %s/10 by %s viewers.
This %s-minute %s movie %s.

Story: %s %s

Movie Profile: This %s film features %s themes. The movie was produced by 
%s in %s with dialogue in %s. 
Popularity score: %s. %s"""

# Create the 'document' column
documents_df = processed_df.withColumn("document",
    format_string(document_template,
        col("title"),
        col("release_date_year").cast("string"),
        col("genres"),
        col("vote_average").cast("string"),
        col("vote_count").cast("string"),
        col("runtime").cast("string"),
        col("original_language"),
        col("status_description"),
        col("overview"),
        col("tagline"),
        col("genres"),
        col("keywords"),
        col("production_companies"),
        col("production_countries"),
        col("spoken_languages"),
        col("popularity").cast("string"),
        col("adult_content_note")
    )
)

# Show a few example documents
documents_df.select("document").show(5, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
documents_path = "/Users/vijayramaraju/Developer/Data Engineering/movieassist/data/documents"

output_documents = documents_df.select("document")

output_documents.write \
    .mode("overwrite") \
    .text(documents_path)

print(f"Documents saved to {documents_path}")

Documents saved to /Users/vijayramaraju/Developer/Data Engineering/movieassist/data/documents


In [26]:
%pip install --upgrade openai httpx


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec
from pyspark.sql.functions import col, monotonically_increasing_id

# Load environment variables from .env file
load_dotenv()

# --- Configuration ---
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_INDEX_NAME = "movieassist"
embedding_model = "text-embedding-3-small"
dimensions = 1536

# --- Error checking for API keys ---
if not PINECONE_API_KEY:
    raise ValueError("PINECONE_API_KEY not found in .env file.")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found in .env file.")

# --- Initialize Pinecone and OpenAI ---
# Initialize Pinecone
pc = Pinecone(api_key=PINECONE_API_KEY)

# Check if index exists, create it if it doesn't
if PINECONE_INDEX_NAME not in pc.list_indexes().names():
    print(f"Creating new index: {PINECONE_INDEX_NAME}")
    pc.create_index(
        name=PINECONE_INDEX_NAME,
        dimension=dimensions,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print(f"Index {PINECONE_INDEX_NAME} already exists.")

# Connect to the index
index = pc.Index(PINECONE_INDEX_NAME)

# Initialize OpenAI client
try:
    openai_client = OpenAI(api_key=OPENAI_API_KEY)
    print("OpenAI client initialized successfully.")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    raise e


# --- Read Documents ---
documents_path = "/Users/vijayramaraju/Developer/Data Engineering/movieassist/data/documents"
docs_df = spark.read.text(documents_path).withColumnRenamed("value", "document")
# Add a unique ID to each document for upserting into Pinecone
docs_df = docs_df.withColumn("id", monotonically_increasing_id().cast("string"))


# --- Function to generate embeddings and upsert ---
def embed_and_upsert(partition):
    # Each partition needs its own client instances
    # Load env vars within the partition function for Spark workers
    load_dotenv()
    partition_pinecone_api_key = os.getenv("PINECONE_API_KEY")
    partition_openai_api_key = os.getenv("OPENAI_API_KEY")

    if not partition_pinecone_api_key or not partition_openai_api_key:
        # If keys are not found, skip processing this partition
        print("API keys not found on worker, skipping partition.")
        return

    openai_client_partition = OpenAI(api_key=partition_openai_api_key)
    pc_partition = Pinecone(api_key=partition_pinecone_api_key)
    index_partition = pc_partition.Index(PINECONE_INDEX_NAME)

    vectors_to_upsert = []
    batch_size = 100 # Pinecone recommends batches of 100 or fewer

    for row in partition:
        try:
            # Generate embedding
            response = openai_client_partition.embeddings.create(
                input=row["document"],
                model=embedding_model
            )
            embedding = response.data[0].embedding



            # Prepare vector for upsert
            vectors_to_upsert.append({
                "id": row["id"],
                "values": embedding,
                "metadata": {"text": row["document"]}
            })

            # Upsert in batches
            if len(vectors_to_upsert) >= batch_size:
                index_partition.upsert(vectors=vectors_to_upsert)
                vectors_to_upsert = []

        except Exception as e:
            print(f"Error processing row {row['id']}: {e}")

    # Upsert any remaining vectors
    if vectors_to_upsert:
        index_partition.upsert(vectors=vectors_to_upsert)

# --- Process and Upsert ---
print("Starting embedding and upsert process...")
# Use foreachPartition to apply the function to each partition of the DataFrame
docs_df.foreachPartition(embed_and_upsert)
print("Embedding and upsert process completed.")

# You can check the status of your index
print(index.describe_index_stats())

Index movieassist already exists.
Error initializing OpenAI client: Client.__init__() got an unexpected keyword argument 'proxies'


TypeError: Client.__init__() got an unexpected keyword argument 'proxies'